<a href="https://colab.research.google.com/github/Tuevu110405/PEFT-LLama-Fine-Tuning-FOR-MCQA/blob/main/Project_PEFT_Llama_Mcqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing libs and datasets

In [ ]:
!pip install -q unsloth==2025.3.15 datasets==3.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.6/195.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset


ds = load_dataset("openlifescienceai/medmcqa")
del ds['test']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/85.9M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/936k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

# Preprocessing

In [ ]:
SYSTEM_PROMPT = "You are a helpful medical assistant specialized in answering multiple-choice questions. Provide the correct answer by only stating the option letter ."

In [ ]:
id2label = {
    0: "A",
    1: "B",
    2: "C",
    3: "D",
}

def formatting_prompt(examples):
    questions = examples["question"]
    opas = examples["opa"]
    opbs = examples["opb"]
    opcs = examples["opc"]
    opds = examples["opd"]
    cops = examples["cop"]

    texts = []
    for idx in range(len(questions)):
        question = questions[idx]
        opa = opas[idx]
        opb = opbs[idx]
        opc = opcs[idx]
        opd = opds[idx]
        answer = id2label[cops[idx]]

        if answer == "A":
            answer = answer
        elif answer == "B":
            answer = answer
        elif answer == "C":
            answer = answer
        elif answer == "D":
            answer = answer


        choices = f"A. {opa}. B. {opb}. C.{opc}. D. {opd}."
        #Build user message content
        user_message_content = f"""

### Question:
{question}

### Choice:
{choices}

"""
        full_conversation = (
            f"<|begin_of_text|>"
            f"<|start_header_id|>system<|end_header_id|>\n{SYSTEM_PROMPT}<|eot_id|>"
            f"<|start_header_id|>user<|end_header_id|>\n{user_message_content}<|eot_id|>"
            f"<|start_header_id|>assistant<|end_header_id|>\n{answer}<|eot_id|>"

        )


        texts.append(full_conversation)
    return {"text": texts,}

process_ds = ds.map(formatting_prompt, batched=True)

Map:   0%|          | 0/182822 [00:00<?, ? examples/s]

Map:   0%|          | 0/4183 [00:00<?, ? examples/s]

In [ ]:
process_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'text'],
        num_rows: 182822
    })
    validation: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'text'],
        num_rows: 4183
    })
})

In [ ]:
process_ds['train'][4]

{'id': 'dc6794a3-b108-47c5-8b1b-3b4931577249',
 'question': 'Growth hormone has its effect on growth through?',
 'opa': 'Directly',
 'opb': 'IG1-1',
 'opc': 'Thyroxine',
 'opd': 'Intranuclear receptors',
 'cop': 1,
 'choice_type': 'single',
 'exp': "Ans. is 'b' i.e., IGI-1GH has two major functions :-i) Growth of skeletal system :- The growth is mediated by somatomedins (IGF). Increased deposition of cailage (including chondroitin sulfate) and bone with increased proliferation of chondrocytes and osteocytes.ii) Metabolic effects :- Most of the metabolic effects are due to direct action of GH. These include gluconeogenesis, decreased peripheral utilization of glucose (decreased uptake), lipolysis and anabolic effect on proteins.",
 'subject_name': 'Physiology',
 'topic_name': None,
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are a helpful medical assistant specialized in answering multiple-choice questions. Provide the correct answer by only stating the op

# Modeling

In [ ]:
import os
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"


In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    dtype= None,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    target_modules =[
        "q_proj","k_proj", "v_proj", "o_proj",
        "down_proj", "up_proj", "gate_proj"
    ],
    use_rslora = True,
    use_gradient_checkpointing = True,
    random_state = 42,

)

print(model.print_trainable_parameters())

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.15: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.3.15 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039
None


# Training

In [ ]:
#Disable wandb

os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

args = TrainingArguments(
    output_dir = "med-mcqa-llama-3.2-1B-4bit-lora",
    logging_dir = "logs",
    learning_rate = 3e-4,
    lr_scheduler_type = "linear",
    per_device_train_batch_size = 16,
    gradient_accumulation_steps = 16,
    num_train_epochs = 2,
    eval_strategy = "steps",
    save_strategy = "steps",
    logging_strategy = "steps",
    eval_steps = 500,
    save_steps = 500,
    logging_steps = 500,
    save_total_limit = 1,
    load_best_model_at_end = True,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.01,
    warmup_steps = 10,
    seed = 0,
    report_to = None,

)





Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer = tokenizer ,
    args = args,
    train_dataset = process_ds["train"],
    eval_dataset = process_ds["validation"],
    dataset_text_field = "text"
)



Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/182822 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/4183 [00:00<?, ? examples/s]

In [ ]:
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 182,822 | Num Epochs = 2 | Total steps = 1,430
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 16 x 1) = 256
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
500,5.561700,4.634082


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


KeyboardInterrupt: 

# Save checkpoints

In [ ]:
from huggingface_hub import login

login(token = "your id")

model.save_pretrained("unsloth-llama-trained")
PEFT_MODEL = "tientuevu/Llama-3.2-1B-bnb-4bit-MedMCQA"
model.push_to_hub(PEFT_MODEL, use_auth_token=True)

README.md:   0%|          | 0.00/580 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Saved model to https://huggingface.co/tientuevu/Llama-3.2-1B-bnb-4bit-MedMCQA


#Inference

 • Option 1:- Sử dụng FastLanguageModel.from_pretrained() để tải mô hình đã huấn luyện với các
 tham số cấu hình như max_seq_length và load_in_4bit.- FastLanguageModel.for_inference(model) giúp tối ưu hóa mô hình cho quá trình suy
 luận (inference).- Dữ liệu đầu vào được tokenized bằng tokenizer, sau đó mô hình sinh kết quả dựa trên
 generate().- Kết quả được giải mã bằng tokenizer.decode().

 • Option 2:- Sử dụng pipeline("text-generation") trong thư viện transformers để tạo ra một pipeline
 sẵn có cho tác vụ sinh văn bản.- Dữliệu đầu vào đượcxửlýbằngpipeline và mô hình sẽ trả về kết quả dựa trên max_new_tokens.- Kết quả trả về dưới dạng chuỗi văn bản sinh ra từ mô hình

# Inference

In [ ]:
from transformers import pipeline
from unsloth import FastLanguageModel
PEFT_MODEL = "tientuevu/Llama-3.2-1B-bnb-4bit-MedMCQA"

#option 1
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = PEFT_MODEL,
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    dtype= None,
)

model = FastLanguageModel.for_inference(model)
model.to("cuda")

input_text = process_ds["validation"][0]["text"]

inputs = tokenizer(input_text, return_tensors = "pt").to("cuda")

output = model.generate(input["input_ids"], max_length = 128)

generated_text = tokenizer.decode(output[0], skip_special_tokens = True)
print(f"Answer: {generated_text}")

#Option 2
generator = pipeline(
    task = "text-generation",
    model = PEFT_MODEL)

output = generator(input_text, max_new_tokens=128, return_full_text = False)[0]
print(f"Answer: {output}")

